In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import log_loss, confusion_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc , precision_score , classification_report
from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import warnings
warnings.filterwarnings('ignore')
import warnings
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('train_PDjVQMB.csv')

In [4]:
df

,ID,Age,Gender,Income,Balance,Vintage,Transaction_Status,Product_Holdings,Credit_Card,Credit_Category,Is_Churn
0,84e2fcc9,36,Female,5L - 10L,563266.44,4,0,1,0,Average,1
1,57fea15e,53,Female,Less than 5L,875572.11,2,1,1,1,Poor,0
2,8df34ef3,35,Female,More than 15L,701607.06,2,1,2,0,Poor,0
3,c5c0788b,43,Female,More than 15L,1393922.16,0,1,2,1,Poor,1
4,951d69c4,39,Female,More than 15L,893146.23,1,1,1,1,Good,1
...,...,...,...,...,...,...,...,...,...,...,...
6645,a8216afd,61,Male,5L - 10L,1354526.19,4,1,2,1,Poor,1
6646,153a306b,42,Female,5L - 10L,596286.54,3,1,2,1,Average,0
6647,d4075a95,29,Male,Less than 5L,979298.19,2,0,2,0,Poor,0
6648,5855b142,66,Female,10L - 15L,395233.65,3,0,2,1,Poor,1


In [5]:
df.dtypes

ID                     object
Age                     int64
Gender                 object
Income                 object
Balance               float64
Vintage                 int64
Transaction_Status      int64
Product_Holdings       object
Credit_Card             int64
Credit_Category        object
Is_Churn                int64
dtype: object

In [6]:
df.Gender.value_counts()

Male      3612
Female    3038
Name: Gender, dtype: int64

In [7]:
df.Income.value_counts()

10L - 15L        1885
5L - 10L         1847
Less than 5L     1573
More than 15L    1345
Name: Income, dtype: int64

In [8]:
df.Credit_Category.value_counts()

Poor       3076
Average    2043
Good       1531
Name: Credit_Category, dtype: int64

In [9]:
df.Is_Churn.value_counts()

0    5113
1    1537
Name: Is_Churn, dtype: int64

In [10]:
df.Product_Holdings.value_counts()

1     3200
2     3182
3+     268
Name: Product_Holdings, dtype: int64

# Label Encoding

In [14]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df.Gender= label_encoder.fit_transform(df.Gender)
 
df.Gender.unique()

array([0, 1])

In [15]:
df.Income = label_encoder.fit_transform(df.Income)
 
df.Income.unique()

array([1, 2, 3, 0])

In [16]:
df.Credit_Category = label_encoder.fit_transform(df.Credit_Category)
 
df.Credit_Category.unique()

array([0, 2, 1])

In [17]:
df.Product_Holdings = label_encoder.fit_transform(df.Product_Holdings)
 
df.Product_Holdings.unique()

array([0, 1, 2])

In [18]:
df =  df.drop('ID',1)
df

,Age,Gender,Income,Balance,Vintage,Transaction_Status,Product_Holdings,Credit_Card,Credit_Category,Is_Churn
0,36,0,1,563266.44,4,0,0,0,0,1
1,53,0,2,875572.11,2,1,0,1,2,0
2,35,0,3,701607.06,2,1,1,0,2,0
3,43,0,3,1393922.16,0,1,1,1,2,1
4,39,0,3,893146.23,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
6645,61,1,1,1354526.19,4,1,1,1,2,1
6646,42,0,1,596286.54,3,1,1,1,0,0
6647,29,1,2,979298.19,2,0,1,0,2,0
6648,66,0,0,395233.65,3,0,1,1,2,1


In [19]:
df.dtypes

Age                     int64
Gender                  int32
Income                  int32
Balance               float64
Vintage                 int64
Transaction_Status      int64
Product_Holdings        int32
Credit_Card             int64
Credit_Category         int32
Is_Churn                int64
dtype: object

In [20]:
X = df.drop('Is_Churn',1)
Y = df.Is_Churn
X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify = Y, test_size = 0.3,random_state=49)

In [21]:
y_train.value_counts()

0    3579
1    1076
Name: Is_Churn, dtype: int64

In [22]:
y_test.value_counts()

0    1534
1     461
Name: Is_Churn, dtype: int64

In [23]:
cols = list(X_train.columns)
cols

['Age',
 'Gender',
 'Income',
 'Balance',
 'Vintage',
 'Transaction_Status',
 'Product_Holdings',
 'Credit_Card',
 'Credit_Category']

In [91]:
scalar = StandardScaler(with_mean=False)
scale_all = scalar.fit_transform(X)
filename = "scalerr"
pickle.dump(scalar,open(filename,"wb"))

In [90]:
scalar = StandardScaler(with_mean=False)
scale_train = pd.DataFrame(scalar.fit_transform(X_train),columns=cols)
scale_test = pd.DataFrame(scalar.transform(X_test),columns=cols)

In [27]:
scale_test

,Age,Gender,Income,Balance,Vintage,Transaction_Status,Product_Holdings,Credit_Card,Credit_Category
0,4.846559,0.000000,0.000000,2.905392,2.757935,2.002177,0.000000,2.107702,2.312362
1,4.743441,0.000000,0.000000,1.691616,3.447418,2.002177,0.000000,2.107702,0.000000
2,5.671505,2.006521,0.000000,0.231751,0.000000,0.000000,1.749357,2.107702,2.312362
3,4.227849,0.000000,0.000000,1.020055,1.378967,2.002177,0.000000,2.107702,0.000000
4,3.402903,2.006521,0.000000,2.581366,1.378967,0.000000,1.749357,2.107702,2.312362
...,...,...,...,...,...,...,...,...,...
1990,5.259032,2.006521,0.911616,1.857763,2.068451,0.000000,0.000000,2.107702,0.000000
1991,3.815376,0.000000,1.823232,0.745221,2.757935,2.002177,0.000000,2.107702,2.312362
1992,4.227849,2.006521,2.734848,0.065786,1.378967,0.000000,1.749357,0.000000,2.312362
1993,5.155914,0.000000,0.911616,1.673185,0.689484,0.000000,1.749357,0.000000,0.000000


In [28]:
final_train_res = scale_train
final_test_res = scale_test

In [95]:
from sklearn.model_selection import RedsdaspeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

ImportError: cannot import name 'RedsdaspeatedStratifiedKFold' from 'sklearn.model_selection' (c:\users\snake\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\__init__.py)

In [68]:
%%time
model = LogisticRegression(class_weight="balanced")
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
class_weight = ['balanced','None']
c_values = [100,10,1.0, 0.1, 0.01, 0.001, ]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values,class_weight=class_weight)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(scale_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.569195 using {'C': 0.001, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'newton-cg'}
0.568570 (0.019777) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'newton-cg'}
0.568570 (0.019777) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'lbfgs'}
0.568570 (0.019777) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}
0.468576 (0.022384) with: {'C': 100, 'class_weight': 'None', 'penalty': 'l2', 'solver': 'newton-cg'}
0.468576 (0.022384) with: {'C': 100, 'class_weight': 'None', 'penalty': 'l2', 'solver': 'lbfgs'}
0.000000 (0.000000) with: {'C': 100, 'class_weight': 'None', 'penalty': 'l2', 'solver': 'liblinear'}
0.568625 (0.019738) with: {'C': 10, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'newton-cg'}
0.568625 (0.019738) with: {'C': 10, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'lbfgs'}
0.568412 (0.019955) with: {'C': 10, 'class_weight': 'balanced', 'penalty': 'l2', 'so

In [75]:
model_log = LogisticRegression(class_weight="balanced", C= 0.001, penalty = "l2", solver = "newton-cg")
model_log.fit(scale_train,y_train)

LogisticRegression(C=0.001, class_weight='balanced', solver='newton-cg')

In [76]:
y_pred = model_log.predict(scale_test)
y_pred

array([1, 1, 1, ..., 0, 1, 1], dtype=int64)

In [83]:
pd.Series(y_pred).value_counts()

0    1140
1     855
dtype: int64

In [84]:
from sklearn.metrics import f1_score
print(f1_score(y_test,y_pred,average='macro'))

0.5646016009166281


In [85]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.62      0.71      1534
           1       0.32      0.59      0.42       461

    accuracy                           0.62      1995
   macro avg       0.58      0.61      0.56      1995
weighted avg       0.72      0.62      0.64      1995



In [93]:
model_log = LogisticRegression(class_weight="balanced", C=0.1, penalty = "l2", solver = "newton-cg")
model_log.fit(scale_all,Y)

LogisticRegression(C=0.1, class_weight='balanced', solver='newton-cg')

In [94]:
import pickle
filename = "logistic.sav"
pickle.dump(model_log, open(filename, 'wb'))

In [104]:
filename = "dt.sav"
pickle.dump(dt_res, open(filename,"wb"))